In [76]:
# Original code from YouTube video: https://www.youtube.com/watch?v=hnAuOakqR90
# Created by: squirrel_plays @ squirrel_plays_fof4318

# Modified by: mythzg91
# Github Repository: https://github.com/mythzg91/FM24-Position-Score-Calculator

# Python 3.10.4


In [77]:
import glob
import os
import pandas as pd

# Check Python Version
import sys
print("Python Version: ", sys.version)

# Check if Python is 3.10.x or above
if sys.version_info[0] != 3 or sys.version_info[1] < 10:
    print("This script requires Python version 3.10.x or above")
    print("Exiting...")
    sys.exit(0)

# Check if libraries are installed
try:
    import pandas as pd
except ImportError:
    print("Pandas is not installed. Please install Pandas and try again.")
    print("Exiting...")
    sys.exit(0)

Python Version:  3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]


In [78]:
# Finds most recent file in specified folder
list_of_files = glob.glob(os.path.join('/path/to/export/folder', '*'))
# Get latest file
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

/users/garyw/Desktop/SQLBackup/FM24 Python/Exported\Shortlist_2027.html


In [79]:
# Read HTML file exported by FM - in this case an example of an output from the squad page
# This reads as a list, not a dataframe
squad_rawdata_list = pd.read_html(latest_file, header=0, encoding="utf-8", keep_default_na=False)

'''
Change your selected position here

Options:

'GoalKeeper'
'CentreBack'
'FullBack'
'WingBack'
'DefensiveMidfielder'
'CentralMidfielder'
'WideMidfielder'
'Winger'
'AttackingMidfielder'
'Striker'

'''

_selected_position = 'Striker'


In [80]:
# turn the list into a dataframe
squad_rawdata = squad_rawdata_list[0]
# Get the column names
squad_rawdata_columns = squad_rawdata.columns

# Check the length of the Column
if len(squad_rawdata_columns) == 0:
    print("No data found. Please check the HTML file and try again.")
    print("Exiting...")
    sys.exit(0)

elif len(squad_rawdata_columns) < 64:
    print("Please use the correct View file.")
    print("Exiting...")
    sys.exit(0)




In [81]:
# English (UK) column names with Extra Attributes
en_header = ['Rec', 'Inf', 'Name', 'Position', 'Nationality', 'Age', 'Club',
       'Transfer Value', 'Wage', 'Min AP', 'Min Fee Rls',
       'Min Fee Rls to Foreign Clubs', 'Personality', 'Media Handling',
       'Left Foot', 'Right Foot', 
       '1v1', 'Acc', 'Aer', 'Agg', 'Agi', 'Ant',
       'Bal', 'Bra', 'Cmd', 'Cnt', 'Cmp', 'Cro', 'Dec', 'Det', 'Dri', 'Fin',
       'Fir', 'Fla', 'Han', 'Hea', 'Jum', 'Kic', 'Ldr', 'Lon', 'Mar', 'OtB',
       'Pac', 'Pas', 'Pos', 'Ref', 'Sta', 'Str', 'Tck', 'Tea', 'Tec', 'Thr',
       'TRO', 'Vis', 'Wor', 
       'Nat', 'Ecc', 'Com', 'Pun', 'Cor', 'Pen', 'L Th', 'Fre', 
       'Height']



# Replace column names with English (UK) column names
# IMPORTANT: Please use the view file in the folder, as the column names are different from the original or other view
squad_rawdata.columns = en_header

# Enums for abbreviation of Position
_position_enum = {
    'GoalKeeper': 'GK',
    'CentreBack': 'D',
    'FullBack': 'D',
    'WingBack': 'WB',
    'DefensiveMidfielder': 'DM',
    'CentralMidfielder': 'M', 
    'WideMidfielder': 'M',
    'Winger': 'AM',
    'AttackingMidfielder': 'AM',
    'Striker': 'ST',
}

# Enums for Position Role
_role_enum = {
    'GoalKeeper': ['GK', 'SK'],
    'CentreBack': ['CD', 'BPD', 'NCB', 'LIB', 'WCB'],
    'FullBack': ['WB', 'FB', 'IFB', 'CWB', 'IWB'],
    'WingBack': ['WB', 'CWB', 'IWB'],
    'DefensiveMidfielder': ['DM', 'DLP', 'BWM', 'A', 'HB', 'RGA', 'RPM', 'VOL'],
    'CentralMidfielder': ['CM', 'DLP', 'BBM', 'BWM', 'AP', 'MEZ', 'CAR', 'RPM'],
    'WideMidfielder': ['WM', 'W', 'IW', 'DW', 'WP'],
    'Winger': ['RMD', 'AP', 'IF', 'IW', 'Tre', 'WTM', 'W'],
    'AttackingMidfielder': ['AM', 'AP', 'Tre', 'SS', 'EG'],
    'Striker': ['DLF', 'AF', 'PF', 'P', 'F9', 'TF', 'CF'],
}   

# Define Location of Attributes
start_loc = '1v1'
end_loc = 'Height'

# If columns data not numeric, example: '1-4', then split text with '-' take the first value then convert to numeric
# start from 1v1 to Height
for i in range(squad_rawdata.columns.get_loc(start_loc), squad_rawdata.columns.get_loc(end_loc)):
    # Convert column data to string
    squad_rawdata.iloc[:, i] = squad_rawdata.iloc[:, i].astype(str)
    
    # If column data is splitable
    if squad_rawdata.iloc[:, i].str.contains('-').any() == True:
       # Split the text with '-', and take the first value. Example: '1-4' will be '1'
       squad_rawdata.iloc[:, i] = squad_rawdata.iloc[:, i].str.split('-').str[0]
       
    # Convert str to numeric
    squad_rawdata.iloc[:, i] = squad_rawdata.iloc[:, i].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
    

# Define Attributes Variables
ATTR_Acceleration = squad_rawdata['Acc'] 
ATTR_Aggression = squad_rawdata['Agg']
ATTR_Agility = squad_rawdata['Agi']
ATTR_Anticipation = squad_rawdata['Ant']
ATTR_Balance = squad_rawdata['Bal']
ATTR_Bravery = squad_rawdata['Bra']
ATTR_Concentration = squad_rawdata['Cnt']
ATTR_Composure = squad_rawdata['Cmp']
ATTR_Crossing = squad_rawdata['Cro']
ATTR_Decisions = squad_rawdata['Dec']
ATTR_Determination = squad_rawdata['Det']
ATTR_Dribbling = squad_rawdata['Dri']
ATTR_Finishing = squad_rawdata['Fin']
ATTR_First_Touch = squad_rawdata['Fir']
ATTR_Flair = squad_rawdata['Fla']
ATTR_Heading = squad_rawdata['Hea']
ATTR_Jumping = squad_rawdata['Jum']
ATTR_Leadership = squad_rawdata['Ldr']
ATTR_Long_Shots = squad_rawdata['Lon']
ATTR_Marking = squad_rawdata['Mar']
ATTR_Off_The_Ball = squad_rawdata['OtB']
ATTR_Pace = squad_rawdata['Pac']
ATTR_Passing = squad_rawdata['Pas']
ATTR_Positioning = squad_rawdata['Pos']
ATTR_Stamina = squad_rawdata['Sta']
ATTR_Strength = squad_rawdata['Str']
ATTR_WorkRate = squad_rawdata['Wor']
ATTR_Tackling = squad_rawdata['Tck']
ATTR_Teamwork = squad_rawdata['Tea']
ATTR_Technique = squad_rawdata['Tec']
ATTR_Vision = squad_rawdata['Vis']
ATTR_WorkRate = squad_rawdata['Wor']
ATTR_Natural_Fitness = squad_rawdata['Nat']
ATTR_Corner = squad_rawdata['Cor']
ATTR_Penalty = squad_rawdata['Pen']
ATTR_Long_Throws = squad_rawdata['L Th']
ATTR_Free_Kicks = squad_rawdata['Fre']

# Goal Keeper Attributes
ATTR_One_On_Ones = squad_rawdata['1v1']
ATTR_Aerial_Reach = squad_rawdata['Aer']
ATTR_Command_Of_Area = squad_rawdata['Cmd']
ATTR_Handling = squad_rawdata['Han']
ATTR_Kicking = squad_rawdata['Kic']
ATTR_Reflexes = squad_rawdata['Ref']
ATTR_Throwing = squad_rawdata['Thr']
ATTR_Rushing_Out = squad_rawdata['TRO']
ATTR_Eccentricity = squad_rawdata['Ecc']
ATTR_Communication = squad_rawdata['Com']
ATTR_Punch_Out = squad_rawdata['Pun']


In [82]:
'''
**Functions to Calculate Essential, Core and Useful Attributes**

Caclulation:
Let's say the position role has 7 Essential attributes, 10 Core attributes and 2 Secondary attributes.
Length of Essential attributes: 7 * 5 = 35
Length of Core attributes: 10 * 3 = 30
Length of Secondary attributes: 2 * 1 = 2

Let's say the max value of each attribute is 20, then the max value of each group is:
Essential attributes: 35 * 20 = 700
Core attributes: 30 * 20 = 600
Secondary attributes: 2 * 20 = 40

The max value of the whole group is 700 + 600 + 40 = 1340

Max value divided by the total length of Essential attributes, Core attributes and Secondary attributes: 
1340 / (35 + 30 + 2) 
= 1340 / 67 
= 20 * 5 
= 100
'''
def Calculate_Scores(*args):
    # Importance of attributes
    essential = 5
    core = 3
    useful = 1

    # Get Variable from function arguments, _Essential_attributes, _Core_attributes, _Secondary_attributes
    Essential_attributes = args[0]
    Core_attributes = args[1]
    Secondary_attributes = args[2]

    # Sum all in the list of Essential_attributes and multiply by essential
    Essential_Score = sum(Essential_attributes) * essential
    # Sum of all Core_attributes and multiply by core
    Core_Score = sum(Core_attributes) * core
    # Sum of all Secondary_attributes and multiply by useful
    Secondary_Score = sum(Secondary_attributes) * useful

    # Get Length of Essential_attributes, Core_attributes, Secondary_attributes
    Essential_attributes_length = len(Essential_attributes) * essential
    Core_attributes_length = len(Core_attributes) * core
    Secondary_attributes_length = len(Secondary_attributes)  * useful

    # Sum of all Essential_attributes_length, Core_attributes_length, Secondary_attributes_length -> Total length
    Total_length = Essential_attributes_length + Core_attributes_length + Secondary_attributes_length

    # Sum of all Essential_Score, Core_Score, Secondary_Score and divide by Total_length then multiply by 5
    Total_Score = ((Essential_Score + Core_Score + Secondary_Score) / Total_length) * 5

    # Return Total_Score
    return Total_Score

In [83]:
# Function to Calculate simple speed and workrate scores
# Calculation: (Number of Attributes) / (Length of Attributes * 20)
def SimpleScores():
    # Speed with Acceleration
    squad_rawdata['Spd'] = ( ATTR_Pace + ATTR_Acceleration ) / 40
    # Change of Direction with Agility
    squad_rawdata['ChgDrt'] = ( ATTR_Agility + ATTR_Balance ) / 40
    # Workrate with Stamina
    squad_rawdata['Work'] = ( ATTR_WorkRate + ATTR_Stamina ) / 40
    # Creativity
    squad_rawdata['Create'] = ( ATTR_Vision + ATTR_Flair + ATTR_Technique ) / 60
    # Passing with Vision and Decisions
    squad_rawdata['Pass'] = ( ATTR_Passing + ATTR_Vision + ATTR_Decisions ) / 60
    # Crossing with Vision and Decisions
    squad_rawdata['Cross'] = ( ATTR_Crossing + ATTR_Vision + ATTR_Decisions ) / 60
    # Receive Pass = Off the ball with Anticipation and First Touch
    squad_rawdata['RecPass'] = ( ATTR_Off_The_Ball + ATTR_Anticipation + ATTR_First_Touch ) / 60
    # Estimated Progression with Age, Determination and Natural Fitness
    # 30 Years old is the peak age, max output value is 2
    squad_rawdata['Progrs'] = ( 30 / squad_rawdata['Age']) * (( ATTR_Determination + ATTR_Natural_Fitness ) / 40)

In [84]:
# Function to Calculate Goal Keeper Scores
def GoalKeeper():
    _Essential_attributes = [
        ATTR_Agility, 
        ATTR_Reflexes
        ]

    _Core_attributes = [
        ATTR_One_On_Ones, 
        ATTR_Command_Of_Area, 
        ATTR_Concentration, 
        ATTR_Kicking, 
        ATTR_Positioning
        ]

    _Secondary_attributes = [
        ATTR_Acceleration, 
        ATTR_Aerial_Reach, 
        ATTR_Composure, 
        ATTR_Decisions, 
        ATTR_First_Touch, 
        ATTR_Handling, 
        ATTR_Passing, 
        ATTR_Throwing, 
        ATTR_Vision
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Function to Calculate Sweeper Keeper Scores
def SweeperKeeper():
    _Essential_attributes = [
        ATTR_Command_Of_Area, 
        ATTR_Kicking, 
        ATTR_One_On_Ones, 
        ATTR_Reflexes, 
        ATTR_Anticipation, 
        ATTR_Concentration, 
        ATTR_Positioning, 
        ATTR_Agility
        ]

    _Core_attributes = [
        ATTR_Aerial_Reach, 
        ATTR_Communication, 
        ATTR_First_Touch, 
        ATTR_Handling, 
        ATTR_Passing, 
        ATTR_Rushing_Out, 
        ATTR_Throwing, 
        ATTR_Composure, 
        ATTR_Decisions, 
        ATTR_Vision, 
        ATTR_Acceleration
        ]

    _Secondary_attributes = [
        ATTR_Eccentricity
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [85]:
# Central Defender
def CentreDefender():
    _Essential_attributes = [
        ATTR_Tackling, 
        ATTR_Positioning
        ]

    _Core_attributes = [
        ATTR_Heading, 
        ATTR_Marking, 
        ATTR_Anticipation, 
        ATTR_Bravery, 
        ATTR_Composure, 
        ATTR_Concentration, 
        ATTR_Decisions, 
        ATTR_Jumping, 
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Aggression, 
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result


# Ball Playing Defender
def BallPlayingDefender():
    _Essential_attributes = [
        ATTR_Passing, 
        ATTR_Tackling, 
        ATTR_Composure, 
        ATTR_Positioning, 
        ATTR_Vision
        ]

    _Core_attributes = [
        ATTR_Heading, 
        ATTR_Marking, 
        ATTR_Technique, 
        ATTR_Anticipation, 
        ATTR_Bravery, 
        ATTR_Concentration, 
        ATTR_Decisions, 
        ATTR_Jumping, 
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_First_Touch, 
        ATTR_Aggression, 
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Libero
def Libero():
    _Essential_attributes = [
        ATTR_First_Touch, 
        ATTR_Marking, 
        ATTR_Passing, 
        ATTR_Tackling, 
        ATTR_Anticipation, 
        ATTR_Composure, 
        ATTR_Concentration, 
        ATTR_Decisions, 
        ATTR_Positioning, 
        ATTR_Teamwork
        ]

    _Core_attributes = [
        ATTR_Dribbling, 
        ATTR_Heading, 
        ATTR_Technique, 
        ATTR_Bravery, 
        ATTR_Flair
        ]

    _Secondary_attributes = [
        ATTR_Acceleration, 
        ATTR_Agility, 
        ATTR_Balance, 
        ATTR_Jumping, 
        ATTR_Pace, 
        ATTR_Stamina, 
        ATTR_Strength, 
        ATTR_Long_Shots, 
        ATTR_Vision
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Non-sense Centre Back
def NonSenseCentreBack():
    _Essential_attributes = [
        ATTR_Tackling, 
        ATTR_Positioning
        ]

    _Core_attributes = [
        ATTR_Heading, 
        ATTR_Marking, 
        ATTR_Anticipation, 
        ATTR_Bravery, 
        ATTR_Concentration, 
        ATTR_Jumping, 
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Aggression, 
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Wide Centre Back
def WideCentreBack():
    _Essential_attributes = [
        ATTR_Crossing, 
        ATTR_Heading, 
        ATTR_Marking, 
        ATTR_Tackling, 
        ATTR_Positioning, 
        ATTR_Jumping, 
        ATTR_Stamina, 
        ATTR_Strength
        ]

    _Core_attributes = [
        ATTR_Dribbling, 
        ATTR_Aggression, 
        ATTR_Bravery, 
        ATTR_Composure, 
        ATTR_Concentration, 
        ATTR_Decisions, 
        ATTR_WorkRate, 
        ATTR_Pace
        ]

    _Secondary_attributes = [
        ATTR_Off_The_Ball
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result


In [86]:
# Function to Calculate Full Back Scores
def FullBack():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_Positioning,
        ]

    _Core_attributes = [
        ATTR_Marking,
        ATTR_Crossing, 
        ATTR_Passing,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Teamwork,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Concentration,
        ]

    _Secondary_attributes = [
        ATTR_WorkRate,
        ATTR_Acceleration,
        ATTR_Balance
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Complete Wing Back
def CompleteWingBack():
    _Essential_attributes = [
        ATTR_Crossing, 
        ATTR_Dribbling, 
        ATTR_First_Touch, 
        ATTR_Passing, 
        ATTR_Technique, 
        ATTR_Decisions, 
        ATTR_Off_The_Ball, 
        ATTR_Teamwork, 
        ATTR_WorkRate, 
        ATTR_Acceleration, 
        ATTR_Pace, 
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Tackling, 
        ATTR_Anticipation, 
        ATTR_Composure, 
        ATTR_Balance, 
        ATTR_Agility
        ]

    _Secondary_attributes = [
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Inverted Wing Back
def InvertedWingBack():
    _Essential_attributes = [
        ATTR_Marking,
        ATTR_Passing,
        ATTR_Tackling,
        ATTR_Decisions,
        ATTR_Teamwork,
        ATTR_WorkRate
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Concentration,
        ATTR_Off_The_Ball,
        ATTR_Acceleration,
        ATTR_Agility,
        ATTR_Stamina,
        ATTR_Anticipation
        ]

    _Secondary_attributes = [
        ATTR_Composure,
        ATTR_Long_Shots,
        ATTR_Flair,
        ATTR_Pace,
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Inverted Full Back
def InvertedFullBack():
    _Essential_attributes = [
        ATTR_Marking,
        ATTR_Tackling,
        ATTR_Anticipation,
        ATTR_Positioning,
        ATTR_Strength
        ]

    _Core_attributes = [
        ATTR_Heading,
        ATTR_Aggression,
        ATTR_Bravery,
        ATTR_Concentration,
        ATTR_Teamwork
        ]

    _Secondary_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Wing Back
def WingBack():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_WorkRate,
        ]

    _Core_attributes = [
        ATTR_Crossing,
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Technique,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Marking,
        ATTR_Positioning,
        ]

    _Secondary_attributes = [
        ATTR_Acceleration,
        ATTR_Balance,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [87]:
# Defensive Midfielder
def DefensiveMidfielder():
    _Essential_attributes = [
        ATTR_Anticipation,
        ATTR_Concentration,
        ATTR_Positioning,
        ATTR_Teamwork,
        ]

    _Core_attributes = [
        ATTR_Marking,
        ATTR_Passing,
        ATTR_Tackling,
        ATTR_Aggression,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_WorkRate,
        ATTR_Stamina,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_First_Touch,
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Deep Lying Playmaker
def DeepLyingPlaymaker():
    _Essential_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Teamwork,
        ATTR_Vision,
        ]

    _Core_attributes = [
        ATTR_Balance,
        ATTR_Anticipation,
        ]

    _Secondary_attributes = [
        ATTR_Tackling,
        ATTR_Off_The_Ball
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result


# Ball Winning Midfielder
def BallWinningMidfielder():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_Aggression,
        ATTR_Anticipation,
        ATTR_Teamwork,
        ATTR_WorkRate
        ]

    _Core_attributes = [
        ATTR_Marking,
        ATTR_Bravery,
        ATTR_Concentration,
        ]

    _Secondary_attributes = [
        ATTR_Passing,
        ATTR_Positioning,
        ATTR_Stamina,
        ATTR_Strength
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result


# Half Back
def HalfBack():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_Marking,
        ATTR_Anticipation,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Positioning
        ]

    _Core_attributes = [
        ATTR_Composure,
        ATTR_Teamwork,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Bravery
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result


# Anchor Man
def AnchorMan():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_Marking,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Positioning,
        ATTR_Teamwork,
        ATTR_Strength
        ]

    _Core_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Aggression,
        ATTR_Bravery,
        ATTR_WorkRate,
        ATTR_Jumping,
        ATTR_Stamina,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Regista
def Regista():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Flair,
        ATTR_Off_The_Ball,
        ATTR_Vision,
        ATTR_Teamwork,
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Long_Shots,
        ATTR_Anticipation,
        ATTR_Balance,
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Acceleration,
        ATTR_Agility
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Roaming Playmaker
def RoamingPlaymaker():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Vision,
        ATTR_Teamwork,
        ATTR_WorkRate,
        ATTR_Acceleration,
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Long_Shots,
        ATTR_Concentration,
        ATTR_Positioning,
        ATTR_Balance,
        ATTR_Agility,
        ATTR_Pace
        ]

    _Secondary_attributes = [
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Volante 
def Volante():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_Tackling,
        ATTR_Off_The_Ball,
        ATTR_Positioning,
        ATTR_WorkRate,
        ATTR_Pace,
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Long_Shots,
        ATTR_Marking,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Acceleration,
        ATTR_Balance,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Flair,
        ATTR_Technique,
        ATTR_Vision
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [88]:
# Advanced Playmaker
def AdvancedPlaymaker():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Vision,
        ATTR_Teamwork
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_Balance
        ]

    _Secondary_attributes = [
        ATTR_Acceleration
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Box to Box Midfielder 
def BoxToBoxMidfielder():
    _Essential_attributes = [
        ATTR_Tackling,
        ATTR_Passing,
        ATTR_Off_The_Ball,
        ATTR_Teamwork,
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Long_Shots,
        ATTR_Technique,
        ATTR_Aggression,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Positioning,
        ATTR_Acceleration,
        ATTR_Balance,
        ATTR_Pace,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Flair,
        ATTR_Agility
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Carrilero 
def Carrilero():
    _Essential_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Tackling,
        ATTR_Decisions,
        ATTR_Positioning,
        ATTR_Teamwork,
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Concentration,
        ATTR_Off_The_Ball,
        ATTR_Vision,
        ATTR_WorkRate
        ]

    _Secondary_attributes = [
        ATTR_Marking
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# =Centre Midfielder
def CentreMidfielder():
    _Essential_attributes = [
        ATTR_Decisions,
        ]

    _Core_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Tackling,
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Teamwork,
        ATTR_WorkRate,
        ATTR_Stamina
        ]

    _Secondary_attributes = [
        ATTR_Long_Shots,
        ATTR_Marking,
        ATTR_Bravery,
        ATTR_Concentration,
        ATTR_Off_The_Ball,
        ATTR_Vision,
        ATTR_Acceleration
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Mezzala 
def Mezzala():
    _Essential_attributes = [
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_WorkRate
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Long_Shots,
        ATTR_Marking,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Vision
        ]

    _Secondary_attributes = [
        ATTR_Acceleration,
        ATTR_Balance,
        ATTR_Stamina,
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [89]:
# Defensive Wide Midfielder
def DefensiveWideMidfielder():
    _Essential_attributes = [
        ATTR_Technique,
        ATTR_Off_The_Ball,
        ATTR_Teamwork,
        ATTR_WorkRate,
        ATTR_Stamina
        ]

    _Core_attributes = [
        ATTR_Crossing,
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Marking,
        ATTR_Tackling,
        ATTR_Aggression,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Positioning,
        ATTR_Acceleration
        ]

    _Secondary_attributes = [
        ATTR_Passing,
        ATTR_Composure,
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Wide Midfielder 
def WideMidfielder():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_Decisions,
        ATTR_Teamwork,
        ATTR_WorkRate
        ]

    _Core_attributes = [
        ATTR_Crossing,
        ATTR_First_Touch,
        ATTR_Tackling,
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Stamina
        ]

    _Secondary_attributes = [
        ATTR_Marking,
        ATTR_Concentration,
        ATTR_Off_The_Ball,
        ATTR_Positioning,
        ATTR_Vision
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

#  Wide Playmaker
def WidePlaymaker():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Vision,
        ATTR_Teamwork
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Off_The_Ball,
        ATTR_Agility
        ]

    _Secondary_attributes = [
        ATTR_Acceleration,
        ATTR_Anticipation,
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result




In [90]:
# Winger 
def Winger():
    _Essential_attributes = [
        ATTR_Off_The_Ball,
        ATTR_Crossing,
        ATTR_Dribbling,
        ATTR_Technique
        ]

    _Core_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ]

    _Secondary_attributes = [
        ATTR_Acceleration,
        ATTR_Agility,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_WorkRate
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Inverted Winger
def InvertedWinger():
    _Essential_attributes = [
        ATTR_Off_The_Ball,
        ATTR_Dribbling,
        ATTR_Passing,
        ATTR_Technique
        ]

    _Core_attributes = [
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Vision,
        ATTR_Crossing,
        ATTR_First_Touch,
        ATTR_Long_Shots,
        ]

    _Secondary_attributes = [
        ATTR_Acceleration,
        ATTR_Agility,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_WorkRate
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Inside Forward
def InsideForward():
    _Essential_attributes = [
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Off_The_Ball,
        ATTR_Acceleration,
        ATTR_Agility,
        ATTR_Balance
        ]

    _Core_attributes = [
        ATTR_Pace,
        ATTR_Finishing,
        ATTR_Passing,
        ATTR_Long_Shots,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Flair
        ]

    _Secondary_attributes = [
        ATTR_Vision
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Raumdeuter
def Raumdeuter():
    _Essential_attributes = [
        ATTR_Finishing,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Balance
        ]

    _Core_attributes = [
        ATTR_First_Touch,
        ATTR_WorkRate,
        ATTR_Acceleration,
        ATTR_Stamina
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Passing
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Wide Target Man
def WideTargetMan():
    _Essential_attributes = [
        ATTR_Heading,
        ATTR_Bravery,
        ATTR_Jumping,
        ATTR_Strength
        ]

    _Core_attributes = [
        ATTR_Off_The_Ball,
        ATTR_Teamwork,
        ATTR_Crossing,
        ATTR_First_Touch,
        ATTR_Anticipation,
        ATTR_WorkRate,
        ATTR_Balance,
        ATTR_Stamina
        ]

    _Secondary_attributes = [
        ATTR_Finishing,
        ATTR_Decisions
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [91]:
# Trequartista
def Trequartista():
    _Essential_attributes = [
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Flair,
        ATTR_Off_The_Ball,
        ATTR_Acceleration,
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_Anticipation,
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Agility,
        ATTR_Balance
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Enganche 
def Enganche():
    _Essential_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Vision,
        ]

    _Core_attributes = [
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_Off_The_Ball,
        ATTR_Teamwork,
        ATTR_Balance
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Agility,
        ATTR_Technique
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Attacking Midfielder 
def AttackingMidfielder():
    _Essential_attributes = [
        ATTR_Passing,
        ATTR_First_Touch,
        ATTR_Long_Shots,
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Decisions,
        ATTR_Flair,
        ATTR_Off_The_Ball
        ]

    _Core_attributes = [
        ATTR_Dribbling,
        ATTR_Composure,
        ATTR_Vision
        ]

    _Secondary_attributes = [
        ATTR_Finishing,
        ATTR_Agility
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Shadow Striker 
def ShadowStriker():
    _Essential_attributes = [
        ATTR_Dribbling,
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Off_The_Ball,
        ATTR_Acceleration,
        ]

    _Core_attributes = [
        ATTR_Passing,
        ATTR_Technique,
        ATTR_Concentration,
        ATTR_Decisions,
        ATTR_WorkRate,
        ATTR_Balance,
        ATTR_Agility,
        ATTR_Pace,
        ATTR_Stamina
        ]

    _Secondary_attributes = [
        ATTR_Flair
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result



In [92]:
# False Nine 
def FalseNine():
    _Essential_attributes = [
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Off_The_Ball,
        ATTR_Technique,
        ATTR_Vision,
        ATTR_Decisions,
        ATTR_Composure,
        ATTR_Acceleration,
        ATTR_Agility
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_Teamwork,
        ATTR_Balance
        ]

    _Secondary_attributes = [
        ATTR_Concentration,
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Poacher
def Poacher():
    _Essential_attributes = [
        ATTR_Finishing,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Off_The_Ball
        ]

    _Core_attributes = [
        ATTR_Acceleration,
        ATTR_Decisions,
        ATTR_Heading,
        ATTR_Technique,
        ATTR_First_Touch
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Strength
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Complete Forward
def CompleteForward():
    _Essential_attributes = [
        ATTR_Dribbling,
        ATTR_First_Touch,
        ATTR_Heading,
        ATTR_Technique,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Off_The_Ball,
        ATTR_Acceleration
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_Long_Shots,
        ATTR_Passing,
        ATTR_Decisions,
        ATTR_Teamwork,
        ATTR_Vision,
        ATTR_WorkRate
        ]

    _Secondary_attributes = [
        ATTR_Agility,
        ATTR_Balance,
        ATTR_Jumping,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Strength
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Deep Lying Forward
def DeepLyingForward():
    _Essential_attributes = [
        ATTR_First_Touch,
        ATTR_Passing,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Teamwork
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_Anticipation,
        ATTR_Flair,
        ATTR_Vision,
        ATTR_Balance,
        ATTR_Strength
        ]

    _Secondary_attributes = [
        ATTR_Dribbling,
        ATTR_Pace
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Target Forward
def TargetForward():
    _Essential_attributes = [
        ATTR_Bravery,
        ATTR_Heading
        ]

    _Core_attributes = [
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Aggression,
        ATTR_Anticipation,
        ATTR_Composure,
        ATTR_Decisions,
        ATTR_Off_The_Ball,
        ATTR_Teamwork
        ]

    _Secondary_attributes = [
        ATTR_Balance,
        ATTR_Jumping,
        ATTR_Strength
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Pressing Forward
def PressingForward():
    _Essential_attributes = [
        ATTR_Acceleration,
        ATTR_Aggression,
        ATTR_Anticipation,
        ATTR_Bravery,
        ATTR_Teamwork,ATTR_WorkRate
        ]

    _Core_attributes = [
        ATTR_First_Touch,
        ATTR_Decisions,
        ATTR_Composure,
        ATTR_Concentration
        ]

    _Secondary_attributes = [
        ATTR_Finishing,
        ATTR_Passing,
        ATTR_Off_The_Ball,
        ATTR_Balance,
        ATTR_Agility,
        ATTR_Pace,
        ATTR_Stamina,
        ATTR_Strength
        ]

    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

# Attacking Forward
def AttackingForward():
    _Essential_attributes = [
        ATTR_Acceleration, 
        ATTR_Dribbling,
        ATTR_Finishing,
        ATTR_First_Touch,
        ATTR_Technique,
        ATTR_Composure,
        ATTR_Off_The_Ball
        ]

    _Core_attributes = [
        ATTR_Passing,
        ATTR_Anticipation,
        ATTR_Decisions,
        ATTR_WorkRate
        ]

    _Secondary_attributes = [
        ATTR_Pace,
        ATTR_Strength,
        ]

        
    result = Calculate_Scores(_Essential_attributes, _Core_attributes, _Secondary_attributes)

    return result

In [93]:
# Execution of each role function based on _selected_position
match _selected_position:
    case 'GoalKeeper':
        squad_rawdata = squad_rawdata.assign(GK=GoalKeeper())
        squad_rawdata = squad_rawdata.assign(SK=SweeperKeeper())
        
    case 'CentreBack':
        squad_rawdata = squad_rawdata.assign(CD=CentreDefender())
        squad_rawdata = squad_rawdata.assign(BPD=BallPlayingDefender())
        squad_rawdata = squad_rawdata.assign(LIB=Libero())
        squad_rawdata = squad_rawdata.assign(NCB=NonSenseCentreBack())
        squad_rawdata = squad_rawdata.assign(WCB=WideCentreBack())

    case 'FullBack':
        squad_rawdata = squad_rawdata.assign(FB=FullBack())
        squad_rawdata = squad_rawdata.assign(CWB=CompleteWingBack())
        squad_rawdata = squad_rawdata.assign(IWB=InvertedWingBack())
        squad_rawdata = squad_rawdata.assign(WB=WingBack())
        squad_rawdata = squad_rawdata.assign(WCB=WideCentreBack())
        squad_rawdata = squad_rawdata.assign(IFB=InvertedFullBack())

    case 'WingBack':
        squad_rawdata = squad_rawdata.assign(CWB=CompleteWingBack())
        squad_rawdata = squad_rawdata.assign(IWB=InvertedWingBack())
        squad_rawdata = squad_rawdata.assign(WB=WingBack())

    case 'DefensiveMidfielder':
        squad_rawdata = squad_rawdata.assign(DM=DefensiveMidfielder())
        squad_rawdata = squad_rawdata.assign(DLP=DeepLyingPlaymaker())
        squad_rawdata = squad_rawdata.assign(BWM=BallWinningMidfielder())
        squad_rawdata = squad_rawdata.assign(HB=HalfBack())
        squad_rawdata = squad_rawdata.assign(A=AnchorMan())
        squad_rawdata = squad_rawdata.assign(RGA=Regista())
        squad_rawdata = squad_rawdata.assign(RPM=RoamingPlaymaker())
        squad_rawdata = squad_rawdata.assign(VOL=Volante())

    case 'CentralMidfielder':
        squad_rawdata = squad_rawdata.assign(CM=CentreMidfielder())
        squad_rawdata = squad_rawdata.assign(MEZ=Mezzala())
        squad_rawdata = squad_rawdata.assign(AP=AdvancedPlaymaker())
        squad_rawdata = squad_rawdata.assign(BBM=BoxToBoxMidfielder())
        squad_rawdata = squad_rawdata.assign(CAR=Carrilero())
        squad_rawdata = squad_rawdata.assign(RPM=RoamingPlaymaker())
        squad_rawdata = squad_rawdata.assign(BWM=BallWinningMidfielder())
        squad_rawdata = squad_rawdata.assign(DLP=DeepLyingPlaymaker())

    case 'WideMidfielder':
        squad_rawdata = squad_rawdata.assign(DW=DefensiveWideMidfielder())
        squad_rawdata = squad_rawdata.assign(WM=WideMidfielder())
        squad_rawdata = squad_rawdata.assign(WP=WidePlaymaker())
        squad_rawdata = squad_rawdata.assign(W=Winger())    
        squad_rawdata = squad_rawdata.assign(IW=InvertedWinger())

    case 'Winger':
        squad_rawdata = squad_rawdata.assign(W=Winger())
        squad_rawdata = squad_rawdata.assign(IW=InvertedWinger())
        squad_rawdata = squad_rawdata.assign(IF=InsideForward())
        squad_rawdata = squad_rawdata.assign(RMD=Raumdeuter())
        squad_rawdata = squad_rawdata.assign(WTM=WideTargetMan())
        squad_rawdata = squad_rawdata.assign(Tre=Trequartista())
        squad_rawdata = squad_rawdata.assign(AP=AdvancedPlaymaker())

    case 'AttackingMidfielder':
        squad_rawdata = squad_rawdata.assign(Tre=Trequartista())
        squad_rawdata = squad_rawdata.assign(EG=Enganche())
        squad_rawdata = squad_rawdata.assign(AM=AttackingMidfielder())
        squad_rawdata = squad_rawdata.assign(SS=ShadowStriker())
        squad_rawdata = squad_rawdata.assign(AP=AdvancedPlaymaker())

    case 'Striker':
        squad_rawdata = squad_rawdata.assign(AF=AttackingForward())
        squad_rawdata = squad_rawdata.assign(CF=CompleteForward())
        squad_rawdata = squad_rawdata.assign(DLF=DeepLyingForward())
        squad_rawdata = squad_rawdata.assign(P=Poacher())
        squad_rawdata = squad_rawdata.assign(F9=FalseNine())
        squad_rawdata = squad_rawdata.assign(TF=TargetForward())
        squad_rawdata = squad_rawdata.assign(PF=PressingForward())
        
    case _:
        # Throw an error if the position is not found
        raise ValueError(f'Position "{_selected_position}" not found')

In [94]:
# Role in _selected_position
_role = _role_enum[_selected_position]
_position = _position_enum[_selected_position]

# Value for _in_position, Example: If player familiar with _position, _in_position = 1.0, else _in_position = 0.25
_in_position_score = 1.0
_not_in_position_score = 0.25

# Add the _in_position column to the DataFrame
squad_rawdata = squad_rawdata.assign(_in_position=_in_position_score)

# Lookup Position in squad_rawdata['Position'], if position doesnt exist then result * _not_in_position_score
position_loc = squad_rawdata.columns.get_loc('Position')
_in_position_loc = squad_rawdata.columns.get_loc('_in_position')

import re

for i in range(len(squad_rawdata)):
    _current = squad_rawdata.iloc[i, position_loc]
    _position_list = []
    regex = r'\(.*\)'
    
    # Split if _position contains ',' or '/'
    if ',' in _current or '/' in _current:
        # Split _current by ',' or '/'
        _position_list = re.split(',|/', _current)
        # Remove 'R', 'L', 'C', '(', ')' from each item in list
        _position_list = [re.sub(regex, '', x) for x in _position_list]
        # Remove space in list
        _position_list = [x.strip() for x in _position_list]

        if _position not in _position_list:
            squad_rawdata.iloc[i, _in_position_loc] = _not_in_position_score
    else:
        # Remove '(','R', 'L', 'C', ')' from each item in _current
        _current = re.sub(regex, '', _current)
        # Remove space in _current
        _current = _current.strip()
        if _position not in _current:
            squad_rawdata.iloc[i, _in_position_loc] = _not_in_position_score
# Role scores * _in_position and Fix to 1 decimal point
for i in _role:
    squad_rawdata[i] = squad_rawdata[i] * squad_rawdata['_in_position'].astype(float).round(1)
    squad_rawdata[i] = squad_rawdata[i].astype(float).round(1)

# Calculate SimpleScores
SimpleScores()

# Convert to float and round to 1 decimal place
simpleScores_columns = [ 'Progrs', 'Spd', 'ChgDrt', 'Work', 'Create', 'Pass', 'Cross', 'RecPass']
for i in simpleScores_columns:
    squad_rawdata[i] = squad_rawdata[i].astype(float).round(1)

# Builds squad dataframe using only columns that will be exported to HTML
squad_table = ['Inf','Name','Age','Club','Transfer Value',
    'Wage','Nationality','Position','Personality','Media Handling',
    'Left Foot', 'Right Foot']

# Add SimpleScores to squad_table
squad_table = squad_table + simpleScores_columns # Comment this line if you dont want to include simpleScores

# Add Height to squad_table
squad_table.append('Height')

# Build squad with _selected_position -> _role_enum[]
squad_table = squad_table + _role

squad = squad_rawdata[squad_table]

In [95]:
# taken from here: https://www.thepythoncode.com/article/convert-pandas-dataframe-to-html-table-python
# creates a function to make a sortable html export

def generate_html(dataframe: pd.DataFrame):
    # get the table HTML from the dataframe
    table_html = dataframe.to_html(table_id="table", index=False)
    # construct the complete HTML with jQuery Data tables
    # You can disable paging or enable y scrolling on lines 20 and 21 respectively
    html = f"""
    <html class="dark">
    <header>
        <link href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css" rel="stylesheet">
        <link href="https://cdn.datatables.net/fixedheader/3.4.0/css/fixedHeader.dataTables.min.css" rel="stylesheet">

    </header>
    <style>
    html {{
        background-color: #1c1e22;
        font-family: Arial, Helvetica, sans-serif;
        color: white;
    }}

    </style>
    <body style="width:100%;">
    <h1 style="text-align: center;">Position: {_selected_position}</h1>
    {table_html}
    <script src="https://code.jquery.com/jquery-3.7.0.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script type="text/javascript" src="https://cdn.datatables.net/fixedheader/3.4.0/js/dataTables.fixedHeader.min.js"></script>

    <script>
        $(document).ready( function () {{
            $('#table').DataTable({{
                paging: false,
                order: [[4, 'desc']],
                fixedHeader: true
                // scrollY: 400,
            }});
            
            // Search bar align left
            $('#table_filter.dataTables_filter').css('float', 'left');
            $('#table_filter.dataTables_filter').css('text-align', 'left');
            $('#table_filter.dataTables_filter').css('padding-bottom', '1rem');


        }});
    </script>
    </body>
    </html>
    """
    # return the html
    return html

In [96]:
# generates random file name for write-out of html file
import uuid

# filename = str(uuid.uuid4()) + ".html"

# Add _selected_position to filename
filename = _selected_position + "_" + str(uuid.uuid4()) + ".html"
filename

'Striker_b18101db-549e-4940-8e4e-d3fe080c9e44.html'

In [97]:
# creates a sortable html export from the dataframe 'squad'

html = generate_html(squad)
open(filename, "w", encoding="utf-8").write(html)

# Output OS + Folder
from os import path

# Get current working directory
cwd = os.getcwd()

# Create folder if not exist
if not path.exists(cwd + '/Output'):
    os.mkdir(cwd + '/Output')

# Move file to output folder
import shutil
shutil.move(cwd + '/' + filename, cwd + '/Output/' + filename)

# Open file in browser
import webbrowser
webbrowser.open_new_tab(cwd + '/Output/' + filename)


True

In [98]:
# Not sure if this path works on macos
# macos_filename = ("/path/to/output/folder" + filename)